# Installing Packages

In [3]:
import sys
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install pandas


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: C:\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: C:\Python311\python.exe -m pip install --upgrade pip


# Importing necessary packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Reading the Dataset

In [3]:
df = pd.read_csv("../dataset/Doctors_job_dataset.csv")

In [4]:
df.head()

,S.No,Resume Title,Work Exp,Annual Salary,Current Location,Preferred Location,Designation,U.G. Course,P. G. Course,Post P. G. Course,Age/Date of Birth,Resume ID,Last Active Date
0,1,Experienced Dermatologist with 5 years plus ex...,5.8,10.0,Kolkata,Kolkata,NaN,Mbbs,Md Dermatology,Dr,40.0,0c264007658bd1c224,2016
1,2,"M.B.B.S.,D.D.V.(Skin) with 5 yrs experience.",5.0,10.5,Dubai,Dubai,Consultant Dermatologist And Cosmetologist,Mbbs,Pg Diploma,NaN,40.0,3697d8852652123c53,2014
2,3,"MBBS, D.V.D & DNB (Dermatology) with 7 yrs exp...",8.2,30.0,Dubai,All India,Dermatologist,Mbbs,Dnb,NaN,48.0,4b98d6db4fc0b5b012,2009
3,4,"MBBS; DNB, FCPS,DDV with 6yrs of Exp.",7.0,9.6,Mumbai,Mumbai,NaN,Mbbs,Dnb Dermatology,D.d.v,41.0,59063066742215cd70,2013
4,5,Medical and cosmetic dermatology and dermatosu...,10.0,15.0,New Delhi,Delhi / Ncr,Consultant Dermatologist And Cosmetologist,Mbbs,Pg Diploma,NaN,40.0,1aea93342d8c652a71,2018


In [5]:
df = df[["Work Exp", "Annual Salary", "Current Location", "Designation","P. G. Course", "Post P. G. Course"]]
df.head()

,Work Exp,Annual Salary,Current Location,Designation,P. G. Course,Post P. G. Course
0,5.8,10.0,Kolkata,NaN,Md Dermatology,Dr
1,5.0,10.5,Dubai,Consultant Dermatologist And Cosmetologist,Pg Diploma,NaN
2,8.2,30.0,Dubai,Dermatologist,Dnb,NaN
3,7.0,9.6,Mumbai,NaN,Dnb Dermatology,D.d.v
4,10.0,15.0,New Delhi,Consultant Dermatologist And Cosmetologist,Pg Diploma,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Work Exp           399 non-null    float64
 1   Annual Salary      399 non-null    float64
 2   Current Location   399 non-null    object 
 3   Designation        361 non-null    object 
 4   P. G. Course       399 non-null    object 
 5   Post P. G. Course  66 non-null     object 
dtypes: float64(2), object(4)
memory usage: 18.8+ KB


In [7]:
df["Designation"].value_counts()

Designation
Consultant Dermatologist                               62
Consultant                                             18
Dermatologist                                          14
Assistant Professor                                    11
Senior Resident                                         9
                                                       ..
Integrator Medical Protocols                            1
Associate Director - Global Aggregate Safety            1
Assistant General Manager, Torrent Research Centre      1
Manager                                                 1
Senior Dermatologist,trichologist And Cosmetologist     1
Name: count, Length: 195, dtype: int64

In [8]:
df["Current Location"].value_counts()

Current Location
Mumbai         58
New Delhi      56
Bangalore      52
Hyderabad      38
Gurgaon        18
               ..
Thrissur        1
Salem           1
Bhubaneswar     1
Kurukshetra     1
Behror          1
Name: count, Length: 80, dtype: int64

In [9]:
df["P. G. Course"].value_counts()

P. G. Course
Medical-ms/md             137
Pg Diploma                 82
Md                         37
Not Pursuing Pg            21
Mba/pgdm                   15
                         ... 
Md Physiology               1
Md Community Medicine       1
M.d Pathology               1
M.d.pharmacology            1
Md Skin                     1
Name: count, Length: 71, dtype: int64

In [10]:
df["Post P. G. Course"].isna()

0      False
1       True
2       True
3      False
4       True
       ...  
394     True
395     True
396     True
397     True
398     True
Name: Post P. G. Course, Length: 399, dtype: bool

In [11]:
def shorten_designation(categories, cutoff):
    categorical_map = {}
    for i in range(len(categories)):
        if categories.values[i] >= cutoff:
            categorical_map[categories.index[i]] = categories.index[i]
        else:
            categorical_map[categories.index[i]] = 'Other'
    return categorical_map

def shorten_location(categories, cutoff):
    location_map = {}
    for i in range(len(categories)):
        if categories.values[i] >= cutoff:
            location_map[categories.index[i]] = categories.index[i]
        else:
            location_map[categories.index[i]] = 'Other'
    return location_map

def clean_PG(x):
    if 'Md' in x or "md" in x or 'm.d' in x or 'M.d.' in x or "MD" in x:
        return "MD"
    if 'Diploma' in x or 'diploma' in x:
        return "Diploma"
    if 'pgdm' in x or "PGDM" in x:
        return "PGDM"
    return "other"

def clean_PostPG(x):
    if pd.isna(x):
        return "No Post PG"
    return "Yes Post PG"

In [12]:
designation_map = shorten_designation(df.Designation.value_counts(), 10)
df['Designation'] = df['Designation'].map(designation_map)
df.Designation.value_counts()

Designation
Other                       256
Consultant Dermatologist     62
Consultant                   18
Dermatologist                14
Assistant Professor          11
Name: count, dtype: int64

In [13]:
location_map = shorten_location(df['Current Location'].value_counts(), 15)
df['Current Location'] = df['Current Location'].map(location_map)
df['Current Location'].value_counts()

Current Location
Other        145
Mumbai        58
New Delhi     56
Bangalore     52
Hyderabad     38
Gurgaon       18
Kolkata       17
Chennai       15
Name: count, dtype: int64

In [14]:
df['P. G. Course'] = df['P. G. Course'].apply(clean_PG)

In [15]:
df['P. G. Course'].value_counts()

P. G. Course
MD         213
Diploma     86
other       85
PGDM        15
Name: count, dtype: int64

In [16]:
df['Post P. G. Course'] = df['Post P. G. Course'].apply(clean_PostPG)

In [17]:
df['Post P. G. Course'].value_counts()

Post P. G. Course
No Post PG     333
Yes Post PG     66
Name: count, dtype: int64

In [18]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
le_designatiom = LabelEncoder()
le_PG = LabelEncoder()
le_PostPG = LabelEncoder()

In [19]:
le_location = LabelEncoder()
df['Current Location'] = le_location.fit_transform(df['Current Location'])
df['Current Location'].unique()

array([4, 7, 5, 6, 1, 2, 3, 0])

In [20]:
df['Designation'] = le_designatiom.fit_transform(df['Designation'])
df['Designation'].unique()

array([5, 4, 3, 2, 1, 0])

In [21]:
df['P. G. Course'] = le_PG.fit_transform(df['P. G. Course'])
df['P. G. Course'].unique()

array([1, 0, 3, 2])

In [22]:
df['Post P. G. Course'] = le_PostPG.fit_transform(df['Post P. G. Course'])
df['Post P. G. Course'].unique()

array([1, 0])

In [23]:
X = df.drop(['Annual Salary'], axis=1)
Y = df['Annual Salary']


In [24]:
from sklearn.model_selection import train_test_split
standard=StandardScaler()
X=standard.fit_transform(X)
# Split the data into train and test data 
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.20,random_state=220)
Y

0      10.0
1      10.5
2      30.0
3       9.6
4      15.0
       ... 
394    27.0
395    18.0
396     6.0
397    30.0
398    18.5
Name: Annual Salary, Length: 399, dtype: float64

In [25]:
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

LinearRegression()

In [26]:
y_pred = linear_reg.predict(X_test)

In [27]:
y_pred

array([20.46589527, 15.41474732, 24.38194309, 15.9407164 , 13.92739344,
       12.41388525, 31.02428411, 22.08705333, 36.7558827 , 34.26104147,
       10.1684546 , 14.44169389,  9.71504253, 15.10769645, 10.00342684,
       14.41868926,  6.93236762, 15.59651282, 13.91572481, 18.73939801,
        8.94691143, 13.37096958, 13.68580836, 13.3148503 , 21.03710482,
       10.41855863, 14.2067314 , 12.72720302, 10.32214634, 11.71791772,
       19.22125403,  6.20868717, 13.61212377,  6.38761238, 14.83061055,
       15.73853594, 11.61334732, 25.4318916 , 21.04406516, 10.95690415,
       19.98433925, 12.6979315 , 12.95656884, 13.62190117, 16.11075244,
       12.8657763 , 17.25360273, 19.1333607 , 13.29184566, 13.77370169,
       25.56695437, 10.21256765, 13.55200403, 16.01562184, 12.66329583,
        7.07927944, 15.74480283, 22.99497872, 14.52982656, 21.16935054,
       25.76943484, 16.92354723, 15.16533675, 33.01557283,  8.45204618,
       13.91572481, 14.8236502 , 13.6433472 , 19.37650439,  7.20

In [28]:
y_test

272    16.8
62     18.0
86     44.8
374     0.0
189     7.2
       ... 
285    12.0
132    30.0
335     1.0
150     0.0
140    13.0
Name: Annual Salary, Length: 80, dtype: float64

In [29]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
error = np.sqrt(mean_squared_error(y_pred, y_test))


In [30]:
error

14.660345271835721

In [31]:
from sklearn.tree import DecisionTreeRegressor
dec_tree_reg = DecisionTreeRegressor(random_state=1)
dec_tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(random_state=1)

In [32]:
y_pred = dec_tree_reg.predict(X_test)

In [33]:
error = np.sqrt(mean_squared_error(y_pred, y_test))

In [34]:
error

18.447614708614772

In [35]:
X

array([[-0.76578441, -0.28001609,  1.28645271, -0.24287039,  2.24620893],
       [-0.92132904,  0.92566471,  0.42666271, -1.22171165, -0.44519456],
       [-0.29915052,  0.92566471, -0.43312729,  1.71481213, -0.44519456],
       ...,
       [ 0.43968646, -0.68190969,  0.42666271, -0.24287039, -0.44519456],
       [-0.5907967 ,  0.52377111, -1.2929173 , -0.24287039, -0.44519456],
       [-0.53246747, -0.28001609,  0.42666271, -0.24287039, -0.44519456]])

In [36]:
x = np.array([[1,'Kolkata','Dermatologist', 'MD', 'Yes Post PG']])
x

array([['1', 'Kolkata', 'Dermatologist', 'MD', 'Yes Post PG']],
      dtype='<U13')

In [37]:
x[:,2] = le_designatiom.transform(x[:,2])

In [38]:
x[:, 1] = le_location.transform(x[:,1])

In [39]:
x[:,3] = le_PG.transform(x[:,3])

In [40]:
x[:,4] = le_PostPG.transform(x[:,4])

In [41]:
x = x.astype(float)

In [42]:
ans = linear_reg.predict(x)

In [43]:
ans

array([30.50878342])

In [45]:
import pickle
with open('salary_pred.pickle','wb') as f:
    pickle.dump(linear_reg,f)